In [27]:
from typing import *
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
# import seaborn as sns
import pandas as pd
# sns.set_style("whitegrid")
import requests

import math
import torch
from torch import nn, Tensor
from torch.nn.functional import softplus
from torch.distributions import Distribution


from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from functools import reduce
from torch.distributions.bernoulli import Bernoulli
from torch import optim

import gzip
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import h5py
import re

## Helper functions

In [10]:
def load_data_chunk(filename, chunk_size=1000):
    """ Load a chunk of data from a gzipped TSV file. """
    return pd.read_csv(filename, sep='\t', compression='gzip', chunksize=chunk_size)

def separate_ids_and_data(data):
    ids = data.iloc[:, 0]
    data = data.iloc[:, 1:]
    return ids, data

class ReparameterizedDiagonalGaussian(Distribution):
    """
    A distribution `N(y | mu, sigma I)` compatible with the reparameterization trick given `epsilon ~ N(0, 1)`.
    """
    def __init__(self, mu: Tensor, log_sigma:Tensor):
        assert mu.shape == log_sigma.shape, f"Tensors `mu` : {mu.shape} and ` log_sigma` : {log_sigma.shape} must be of the same shape"
        self.mu = mu
        self.sigma = log_sigma.exp()

    def sample_epsilon(self) -> Tensor:
        """`\eps ~ N(0, I)`"""
        return torch.empty_like(self.mu).normal_()

    def sample(self) -> Tensor:
        """sample `z ~ N(z | mu, sigma)` (without gradients)"""
        with torch.no_grad():
            return self.rsample()

    def rsample(self) -> Tensor:
        """sample `z ~ N(z | mu, sigma)` (with the reparameterization trick) """
        # z = mu + sigma * epsilon
        return self.mu + self.sigma * self.sample_epsilon()

    def log_prob(self, z: Tensor) -> Tensor:
        """return the log probability: log `p(z)`"""
        # Log probability for Gaussian distribution
        # log p(z) = -1/2 * [log(2*pi) + 2*log(sigma) + (z - mu)^2/sigma^2]
        return -0.5 * (torch.log(2 * torch.tensor(math.pi)) + 2 * torch.log(self.sigma) +
                       torch.pow(z - self.mu, 2) / torch.pow(self.sigma, 2))
    
    def count_csv_rows(filename):
        # If the file is gzip-compressed, decompress it first
        if filename.endswith('.gz'):
            with gzip.open(filename, 'rt', newline='') as csvfile:
                row_count = sum(1 for row in csvfile)
        else:
            # Specify the correct encoding (e.g., 'utf-8', 'latin-1', etc.)
            encoding = 'utf-8'  # Change to the appropriate encoding if needed
            with open(filename, 'r', newline='', encoding=encoding) as csvfile:
                row_count = sum(1 for row in csvfile)
        return row_count

## Load Data

In [11]:
# Define the file paths
archs4_path = "/dtu-compute/datasets/iso_02456/archs4_gene_expression_norm_transposed.tsv.gz"
gtex_gene_path = "/dtu-compute/datasets/iso_02456/gtex_gene_expression_norm_transposed.tsv.gz"
gtex_isoform_path = "/dtu-compute/datasets/iso_02456/gtex_isoform_expression_norm_transposed.tsv.gz"
gtex_anno_path = "/dtu-compute/datasets/iso_02456/gtex_gene_isoform_annoation.tsv.gz"
gtex_tissue_path = "/dtu-compute/datasets/iso_02456/gtex_annot.tsv.gz"

num_genes = 18965
num_isoforms = 156958

num_genes, num_isoforms

(18965, 156958)

In [12]:
# count_csv_rows(gtex_gene_path)
print("gtex_gene_path num rows: 17357")
percentage_calc = 17357 * 0.0005
percentage_calc

gtex_gene_path num rows: 17357


8.6785

Using percentage 0.001 should load 17 samples, and 0.01 170. We use 0.005 for 86 samples

### Data Loader

- Label Encoding: This converts each unique label into a unique integer. If you have a large number of classes, be aware of memory usage.
- Inverse Transformation: If you need to get back the original labels from the encoded ones, you can use self.label_encoder.inverse_transform().
- Data Types: Labels are converted to torch.long since they are now integers, which is a common practice for categorical labels in classification tasks.

In [13]:
GTEX_NUM_ROWS = 17356

class GeneExpressionDataset(Dataset):
    """ # Old dataLoader with no TestSet
    def __init__(self, filepath, percentage=0.1):
        self.data = self.load_data_percentage(filepath, percentage)
        # Assuming the first column is the label
        self.labels = self.data.iloc[:, 0]
        self.genes = self.data.iloc[:, 1:]

        # Encode the labels
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(self.labels)
    """

    def __init__(self, data, labels):
        self.labels = labels
        self.genes = data
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.genes)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        label = self.encoded_labels[idx]
        genes = self.genes.iloc[idx]
        return torch.tensor(genes.values, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

    def get_original_labels(self, encoded_labels):
        return self.label_encoder.inverse_transform(encoded_labels)

    # Next function is slow because it counts all the rows to calculate the percentage, that is why we added load rows
    @staticmethod
    def load_data_percentage(filepath, percentage=0.1):
        # Load a percentage of the data as shown previously
        cols = pd.read_csv(filepath, sep='\t', compression='gzip', nrows=0).columns
        n_total_rows = sum(1 for row in open(filepath, 'rb'))
        n_rows_to_load = int(n_total_rows * percentage)
        skip_rows = np.random.choice(np.arange(1, n_total_rows), size=n_total_rows - n_rows_to_load, replace=False)
        return pd.read_csv(filepath, sep='\t', compression='gzip', usecols=cols, skiprows=skip_rows)
    
    @staticmethod
    def load_rows(filepath, num_rows):
        """
        Load a specific number of rows from the file.
        """
        return pd.read_csv(filepath, sep='\t', compression='gzip', nrows=num_rows)

class GtexDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir:str="/dtu-compute/datasets/iso_02456/hdf5/", include:str="", exclude:str="", load_in_mem:bool=False):
        f_gtex_gene = h5py.File(data_dir + 'gtex_gene_expression_norm_transposed.hdf5', mode='r')
        f_gtex_isoform = h5py.File(data_dir + 'gtex_isoform_expression_norm_transposed.hdf5', mode='r')

        self.dset_gene = f_gtex_gene['expressions']
        self.dset_isoform = f_gtex_isoform['expressions']

        assert(self.dset_gene.shape[0] == self.dset_isoform.shape[0])

        if load_in_mem:
            self.dset_gene = np.array(self.dset_gene)
            self.dset_isoform = np.array(self.dset_isoform)

        self.idxs = None

        if include and exclude:
            raise ValueError("You can only give either the 'include_only' or the 'exclude_only' argument.")

        if include:
            matches = [bool(re.search(include, s.decode(), re.IGNORECASE)) for s in f_gtex_gene['tissue']]
            self.idxs = np.where(matches)[0]

        elif exclude:
            matches = [not(bool(re.search(exclude, s.decode(), re.IGNORECASE))) for s in f_gtex_gene['tissue']]
            self.idxs = np.where(matches)[0]

    def __len__(self):
        if self.idxs is None:
            return self.dset_gene.shape[0]
        else:
            return self.idxs.shape[0]

    def __getitem__(self, idx):
        if self.idxs is None:
            return self.dset_gene[idx], self.dset_isoform[idx]
        else:
            return self.dset_gene[self.idxs[idx]], self.dset_isoform[self.idxs[idx]]
        
class PartialDataset(Dataset):
    def __init__(self, data: Dataset, start: int, end: int):
        assert start >= 0
        assert end < len(data)

        self.data = data
        self.start = start
        self.end = end
    
    def __len__(self):
        return self.end - self.start + 1

    def __getitem__(self, idx):
        return self.data[self.start + idx]

    


In [15]:
num_rows_to_load = 100  # specify the number of rows you want to load

# Print progress
print("Loading data...")

# Load the entire dataset
full_data = GtexDataset()
# Print progress
print("Data loaded.")

# Split the data into training and testing sets
print("Splitting data into training and testing sets...")
gene_train_dataset = PartialDataset(full_data, 0, 500)
gene_test_dataset = PartialDataset(full_data, 501, 520)

# Print progress
print("Data split.")

# # Create dataset instances for training and testing
# print("Creating training and testing datasets...")
# gene_train_dataset = GeneExpressionDataset(train_data.iloc[:, 1:], train_data.iloc[:, 0])
# gene_test_dataset = GeneExpressionDataset(test_data.iloc[:, 1:], test_data.iloc[:, 0])

# Print progress
print("Datasets created.")

gene_train_loader = DataLoader(gene_train_dataset, batch_size=64, shuffle=True)
gene_test_loader = DataLoader(gene_test_dataset, batch_size=64, shuffle=False)  # Usually, shuffling is not needed for testing

# Print progress
print("Data loaders created.")

Loading data...
Data loaded.
Splitting data into training and testing sets...
Data split.
Datasets created.
Data loaders created.


In [16]:
# Same code with percentage and without showing progress 
"""
load_percentage = 0.0005

# Load the entire dataset
full_data = GeneExpressionDataset.load_data_percentage(gtex_gene_path, percentage=load_percentage)

# Split the data into training and testing sets
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)

# Create dataset instances for training and testing
gene_train_dataset = GeneExpressionDataset(train_data.iloc[:, 1:], train_data.iloc[:, 0])
gene_test_dataset = GeneExpressionDataset(test_data.iloc[:, 1:], test_data.iloc[:, 0])

gene_train_loader = DataLoader(gene_train_dataset, batch_size=64, shuffle=True)
gene_test_loader = DataLoader(gene_test_dataset, batch_size=64, shuffle=False)  # Usually, shuffling is not needed for testing
"""


'\nload_percentage = 0.0005\n\n# Load the entire dataset\nfull_data = GeneExpressionDataset.load_data_percentage(gtex_gene_path, percentage=load_percentage)\n\n# Split the data into training and testing sets\ntrain_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)\n\n# Create dataset instances for training and testing\ngene_train_dataset = GeneExpressionDataset(train_data.iloc[:, 1:], train_data.iloc[:, 0])\ngene_test_dataset = GeneExpressionDataset(test_data.iloc[:, 1:], test_data.iloc[:, 0])\n\ngene_train_loader = DataLoader(gene_train_dataset, batch_size=64, shuffle=True)\ngene_test_loader = DataLoader(gene_test_dataset, batch_size=64, shuffle=False)  # Usually, shuffling is not needed for testing\n'

In [17]:
full_data

In [18]:
genes, labels = next(iter(gene_train_loader))
genes, labels

(tensor([[2.1827, 3.0108, 0.9561,  ..., 0.2750, 1.5801, 0.6041],
         [1.5410, 3.1210, 0.1506,  ..., 0.0000, 2.4249, 0.0000],
         [0.6135, 0.7740, 0.0000,  ..., 0.0000, 1.0772, 0.0426],
         ...,
         [3.1985, 2.0426, 0.0976,  ..., 0.0426, 1.6229, 0.0000],
         [3.3219, 1.8836, 1.3785,  ..., 0.0000, 0.4114, 0.0841],
         [1.8992, 3.7876, 0.1110,  ..., 0.0000, 0.2750, 0.0426]]),
 tensor([[0.4436, 0.0000, 0.6041,  ..., 2.6645, 2.6182, 0.0000],
         [0.0000, 0.0000, 1.3448,  ..., 2.5311, 2.4751, 0.0000],
         [0.3561, 0.0000, 4.0652,  ..., 0.0000, 0.4854, 0.0000],
         ...,
         [0.0000, 0.0000, 0.5558,  ..., 0.6690, 1.8953, 0.0000],
         [0.9635, 0.0000, 0.9855,  ..., 1.0356, 3.1473, 1.2750],
         [0.2016, 0.4436, 0.9635,  ..., 1.8875, 3.1619, 0.3334]]))

In [19]:
test_genes, test_labels = next(iter(gene_test_loader))
test_genes, test_labels

(tensor([[4.1993, 3.7506, 0.0000,  ..., 0.0000, 0.8639, 0.0000],
         [2.1890, 0.6135, 0.2869,  ..., 0.0000, 1.6915, 0.1110],
         [3.0054, 1.4489, 1.3561,  ..., 0.0566, 0.7312, 0.0000],
         ...,
         [2.8074, 3.0772, 1.0356,  ..., 0.0000, 0.9486, 0.0426],
         [2.6206, 3.4463, 0.2510,  ..., 0.8156, 1.9635, 0.0000],
         [2.5286, 1.5801, 0.9411,  ..., 0.0000, 0.2265, 0.0000]]),
 tensor([[0.0000, 0.0000, 0.0000,  ..., 0.1763, 1.2388, 0.0000],
         [0.5945, 0.0000, 1.2327,  ..., 4.9284, 1.9146, 0.0000],
         [0.0000, 0.0000, 2.0072,  ..., 3.8115, 2.5311, 0.0000],
         ...,
         [0.7570, 0.6871, 0.6041,  ..., 3.2987, 2.3813, 0.8953],
         [0.0000, 0.0000, 1.8914,  ..., 6.3420, 2.9598, 0.4957],
         [0.2750, 1.2265, 0.5753,  ..., 1.2987, 3.3909, 0.0000]]))

# Todo: Add code to load VAE or latent features outputed from the VAE for gtex_gene_path

## Building the model
When defining the model the latent layer must act as a bottleneck of information, so that we ensure that we find a strong internal representation. We initialize the VAE with 1 hidden layer in the encoder and decoder using relu units as non-linearity.

## Training and Evaluation

### Initialize the model, evaluator and optimizer

In [12]:
print(torch.__version__)
print(torch.version.cuda)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> Using device: {device}")

1.10.2+cu102
10.2
>> Using device: cpu


In [13]:
train_loader = gene_train_loader
test_loader = gene_test_loader

In [30]:
NUM_FEATURES = 2000
NUM_OUTPUT = 156958

# define network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        activation_fn = nn.ReLU
        num_hidden = (NUM_FEATURES + NUM_OUTPUT) // 4
        self.net = nn.Sequential(
            nn.Linear(NUM_FEATURES, num_hidden),
            activation_fn(),
            nn.Linear(num_hidden, NUM_OUTPUT),
            activation_fn()
        )

    def forward(self, x):
        return self.net(x)

model = Net()
# device = torch.device('cuda')  # use cuda or cpu
# model.to(device)
print(model)


loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)


# # Test the forward pass with dummy data
# out = model(torch.randn(2, 3, 32, 32, device=device))
# print("Output shape:", out.size())
# print(f"Output logits:\n{out.detach().cpu().numpy()}")
# print(f"Output probabilities:\n{out.softmax(1).detach().cpu().numpy()}")




# batch_size = 64
# num_epochs = 10
# validation_every_steps = 500

# step = 0
# model.train()

# train_accuracies = []
# valid_accuracies = []

# for epoch in range(num_epochs):

#     train_accuracies_batches = []

#     for inputs, targets in train_loader:
#         inputs, targets = inputs.to(device), targets.to(device)

#         # Forward pass, compute gradients, perform one training step.
#         # Your code here!

#         # Forward pass.
#         output = model(inputs)

#         # Compute loss.
#         loss = loss_fn(output, targets)

#         # Clean up gradients from the model.
#         optimizer.zero_grad()

#         # Compute gradients based on the loss from the current batch (backpropagation).
#         loss.backward()

#         # Take one optimizer step using the gradients computed in the previous step.
#         optimizer.step()

#         # Increment step counter
#         step += 1

#         # Compute accuracy.
#         predictions = output.max(1)[1]
#         train_accuracies_batches.append(accuracy(targets, predictions))

#         if step % validation_every_steps == 0:

#             # Append average training accuracy to list.
#             train_accuracies.append(np.mean(train_accuracies_batches))

#             train_accuracies_batches = []

#             # Compute accuracies on validation set.
#             valid_accuracies_batches = []
#             total_loss = 0

#             with torch.no_grad():
#                 model.eval()
#                 for inputs, targets in test_loader:
#                     inputs, targets = inputs.to(device), targets.to(device)
#                     output = model(inputs)
#                     loss = loss_fn(output, targets)
#                     total_loss += loss

#                     predictions = output.max(1)[1]

#                     # Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
#                     valid_accuracies_batches.append(accuracy(targets, predictions) * len(inputs))

#                 model.train()

#             # Append average validation accuracy to list.
#             valid_accuracies.append(np.sum(valid_accuracies_batches) / len(test_set))

#             print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}")
#             print(f"             test accuracy: {valid_accuracies[-1]}")
#             print(f"             total loss: {total_loss}")

# print("Finished training.")

RuntimeError: [enforce fail at alloc_cpu.cpp:83] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 24949415848 bytes. Error code 12 (Cannot allocate memory)

## Evaluate model